#MyJohnDeere API Best Practices#

This notebook contains a variety of 'Best Practices' that we have learned over the years that will help you use the MyJohnDeere APIs to the fullest extent.

**Note - This notebook requires that you be logged into Google.**

Copyright (c) 2018 Deere & Company
 
This software may be modified and distributed under the terms
of the MIT license.  See the LICENSE file for details.

#OAuth 1.0a Code for working with MyJohnDeere API#

The `requests_oauthlib` is a popular Python library for working with OAuth

In [0]:
from requests_oauthlib import OAuth1Session

Using the application you created on https://developer.deere.com, copy the "App Id" and "Shared Secret" and use those to set the `client_key` and `client_secret` variables

![Newly Created App](https://github.com/joshuajcarson/DevelopWithDeereOnboardingPictures/raw/master/NewlyCreatedApp.jpg)

In [0]:
client_key = 'place_client_key_here'
client_secret = 'place_client_secret_here'

#Leave the line below as-is. This line of code verifies that you've modified the client_key and client_secret to the values above so that your application can complete OAuth"
assert(client_key != 'place_client_key_here' and client_secret != 'place_client_secret_here'), "You need to update your client_key and client_secret in this cell"

Used the OAuth 1.0a URLs as found at https://developer.deere.com/#!documentation&doc=myjohndeere%2FoAuth.htm&anchor=

In [0]:
request_url = 'https://sandboxapi.deere.com/platform/oauth/request_token'
authorize_url = 'https://my.deere.com/consentToUseOfData'
access_token_url = 'https://sandboxapi.deere.com/platform/oauth/access_token'

MyJohnDeere API uses a specific header for most types of json requests. We'll use that be default for our API calls

In [0]:
MYJOHNDEERE_V3_JSON_HEADERS = {'Accept': 'application/vnd.deere.axiom.v3+json',
               'Content-Type': 'application/vnd.deere.axiom.v3+json'}

We will now slowly start to create our MyJohnDeere API SDK. For this notebook, we'll focus the functions on exclusively using the MyJohnDeere API.

First, we'll wrap up the logic for dealing with the request token step of OAuth into a couple of functions

In [0]:
def get_authorization_url(resource_owner_key):
  return authorize_url + '?oauth_token=' + resource_owner_key

def get_request_token_information_for_myjohndeere_api():
  oauth_request_token_step = OAuth1Session(client_key, client_secret=client_secret, callback_uri='oob')
  request_token_response = oauth_request_token_step.fetch_request_token(request_url)
  resource_owner_key = request_token_response.get('oauth_token')
  resource_owner_secret = request_token_response.get('oauth_token_secret')
  complete_authorize_url = get_authorization_url(resource_owner_key)
  return resource_owner_key, resource_owner_secret, complete_authorize_url 

After the request token step, we need to begin the verification flow. This is the step where the John Deere customer grants active consent to the software application to see the customer's data. Due to the fact this notebook acts similarly to desktop software, we'll direct the customers over to a specific link that John Deere manages 

In [0]:
request_resource_owner_key, request_resource_owner_secret, complete_authorize_url = get_request_token_information_for_myjohndeere_api()
"Click On " + complete_authorize_url + " and paste the results in the next cell as the verifier variable"

'Click On https://my.deere.com/consentToUseOfData?oauth_token=6fa206c0-021d-4c0c-b2a6-c1d2fc21527f and paste the results in the next cell as the verifier variable'

Follow the link from the results of the cell above, and copy the "Verifier" from that screen into the cell below
![Verifier Screen](https://github.com/joshuajcarson/DevelopWithDeereOnboardingPictures/raw/master/VerifierScreen.jpg)

In [0]:
verifier = 'place_verifier_here'

#Leave the line below as-is. This is to make sure that the cell above is modified so that the verifier step of OAuth can finish.
assert(verifier != 'place_verifier_here'), 'The verifier in this cell must be replaced by the verified that you should get from MyJohnDeere'

Using the `verifier` from the step above, we'll create another function that continues the OAuth process with the verification step

In [0]:
def get_verifier_token_information_for_myjohndeere_api(resource_owner_key, resource_owner_secret, verifier):
  oauth_verifier_step = OAuth1Session(client_key,
                          client_secret=client_secret,
                          resource_owner_key=resource_owner_key,
                          resource_owner_secret=resource_owner_secret,
                          verifier=verifier)
  oauth_tokens = oauth_verifier_step.fetch_access_token(access_token_url)
  resource_owner_key = oauth_tokens.get('oauth_token')
  resource_owner_secret = oauth_tokens.get('oauth_token_secret')
  return resource_owner_key, resource_owner_secret

In [0]:
verifier_resource_owner_key, verifier_resource_owner_secret = get_verifier_token_information_for_myjohndeere_api(request_resource_owner_key, request_resource_owner_secret, verifier)

We'll create a function that encapsilates the final step of the OAuth flow where we establish the completed OAuth session

In [0]:
def get_oauth_session_for_myjohndeere_api(resource_owner_key, resource_owner_secret):
  return OAuth1Session(client_key, client_secret=client_secret, resource_owner_key=resource_owner_key, resource_owner_secret=resource_owner_secret)

In [0]:
oauth_session = get_oauth_session_for_myjohndeere_api(verifier_resource_owner_key, verifier_resource_owner_secret)

#Using `links` within the MyJohnDeere API#

Links are a powerful tool in APIs that support [HATEOAS](https://en.wikipedia.org/wiki/HATEOAS). The MyJohnDeere API is no different. To start with, let us call the `API Catalog` and explore those links

In [0]:
api_catalog_url = 'https://sandboxapi.deere.com/platform/'

api_catalog_response = oauth_session.get(api_catalog_url, headers = MYJOHNDEERE_V3_JSON_HEADERS)

api_catalog_response.json()

While links can be incredible useful, they can often be made more usable to your praticular programming language with a few helper methods. The following function converts from the link JSON object into a python dictionary

In [0]:
def convert_links_array_to_dictionary(links_a):
    link_dict = dict()
    for link_o in links_a:
        key = link_o['rel']
        value = link_o['uri']
        link_dict[key] = value
    return link_dict

Lets look again at the API Catalog, only this time after we change the links from an array of link objects to a dictionary

In [0]:
api_catalog_links_as_dictionary = convert_links_array_to_dictionary(api_catalog_response.json()['links'])
api_catalog_links_as_dictionary

That looks a bit easier to work with. The `rel` has been converted to an attribute and the `uri` is now the value. Let us take it a step farther and modify the object itself to make using the responses themselves a bit easier.

In [0]:
def replace_links_as_object_array_with_links_as_dictionary(object_with_links_to_convert):
  object_with_links_converted = object_with_links_to_convert
  object_with_links_converted['links'] = convert_links_array_to_dictionary(object_with_links_to_convert['links'])
  return object_with_links_converted

In [0]:
api_catalog_with_clean_links = replace_links_as_object_array_with_links_as_dictionary(api_catalog_response.json())
api_catalog_with_clean_links

When calling the MyJohnDeere API for resources, you can control which possible links will appear in the response. This is done via the `showLinks` request param, where the values act as a filter to on the returned links. Lets look at the `API Catalog` again, but with fewer links

In [0]:
api_catalog_showLinks_request_param = {'showLinks': 'currentToken,currentUser,organizations'}

api_catalog_response_with_request_param = oauth_session.get(api_catalog_url, headers = MYJOHNDEERE_V3_JSON_HEADERS, params = api_catalog_showLinks_request_param)
api_catalog_response_with_request_param.json()

Using the `showLinks` request param is helpful once you know which path through the MyJohnDeere API that your application should go through. It will reduce overall network traffic sometimes by more than 50%.

Now lets chain our new helpful functions together to get just the links we want as well as in the format to make them even easier to work with

In [0]:
def create_show_links_request_param(links_to_show_comma_separated = None):
  return {'showLinks' : links_to_show_comma_separated}

def get_myjohndeere_api_json_response(oauth_session, myjohndeere_uri, headers = MYJOHNDEERE_V3_JSON_HEADERS, params = None):
  json_response = oauth_session.get(myjohndeere_uri, headers = headers, params = params).json()
  return replace_links_as_object_array_with_links_as_dictionary(json_response)

In [0]:
api_catalog_response_with_clean_links = get_myjohndeere_api_json_response(oauth_session, myjohndeere_uri = api_catalog_url, params = create_show_links_request_param('currentToken,currentUser,organizations'))
api_catalog_response_with_clean_links

Now that we can manage links well, lets follow a link to discover other resources within the MyJohnDeere API. Let's follow the '`currentUser`' link to see more information about the currently logged in user.

In [0]:
current_user_link = api_catalog_response_with_clean_links['links']['currentUser']
current_user_response = get_myjohndeere_api_json_response(oauth_session, myjohndeere_uri = current_user_link)
current_user_response

#Working with the `Collection` object : counts, nextPage links, and combining results#

Many of the APIs within the MyJohnDeere API return back a list of objects back, as compared to the single object responses we've seen with the `API Catalog` and `currentUser` links. We can start seeing a collection by following the '`organizations`' link from the `currentUser`

In [0]:
organizations_for_current_user_link = current_user_response['links']['organizations']
get_myjohndeere_api_json_response(oauth_session, myjohndeere_uri = organizations_for_current_user_link)

We can see from the response that we get back links for the collection itself, which can contain links for '`self`' as well as the '`nextPage`' and '`previousPage`' used during pagination. We'll go into pagination in a later section. We also see attributes for `total` and `values`. The `total` attribute represents how many objects exist within the collection across all pages. 

The `values` attribute is an array of the actual objects we're interested in. In this particular example, we can see that the `links` array within each object could benefit from benefit from being simplified.

In [0]:
def get_myjohndeere_api_collection_json_response(oauth_session, myjohndeere_uri, headers = MYJOHNDEERE_V3_JSON_HEADERS, params = None):
  collection_json_response = get_myjohndeere_api_json_response(oauth_session, myjohndeere_uri, headers, params)
  values_from_collection = collection_json_response['values']
  values_to_add_back_to_collection = []
  for object in values_from_collection:
    values_to_add_back_to_collection.append(replace_links_as_object_array_with_links_as_dictionary(object))
  collection_json_response['values'] = values_to_add_back_to_collection
  return collection_json_response

In [0]:
organizations_for_current_user_link = current_user_response['links']['organizations']
get_myjohndeere_api_collection_json_response(oauth_session, myjohndeere_uri = organizations_for_current_user_link)

Now that we have simplified the JSON that we're working with, we will next handle pagination. The MyJohnDeere API uses pagination in order to reduce the load per API request. You can learn more about pagination at https://medium.com/square-corner-blog/tips-and-tricks-for-api-pagination-5cacc6f017da.

In order to control the pagination, MyJohnDeere makes uses the matrix parameters `start` and `count`. In order to trigger pagination, we'll set the `count` to one and make our request. 

In [0]:
organizations_for_current_user_link = current_user_response['links']['organizations']
organizations_for_current_user_link_with_count_of_1 = organizations_for_current_user_link + ';count=1'
get_myjohndeere_api_collection_json_response(oauth_session, myjohndeere_uri = organizations_for_current_user_link_with_count_of_1)

If we look closely at the response assuming that your logged in user has two organizations they are a staff member of, we should see a `nextPage` link, which wll have both `start=1` and `count=1`. Lets enhance our last method to follow any `nextPage` link found.

In [0]:
def get_myjohndeere_api_collection_values_for_all_pages(oauth_session, myjohndeere_uri, headers = MYJOHNDEERE_V3_JSON_HEADERS, params = None):
  collection_response = get_myjohndeere_api_collection_json_response(oauth_session, myjohndeere_uri, headers, params)
  collection_links = collection_response['links']
  while 'nextPage' in collection_links:
    next_page_response = get_myjohndeere_api_collection_json_response(oauth_session, collection_links['nextPage'], headers, params)
    collection_links = next_page_response['links']
    collection_response['values'].extend(next_page_response['values'])
  return collection_response['values']

In [0]:
organizations_for_current_user_link = current_user_response['links']['organizations']
organizations_for_current_user_link_with_count_of_1 = organizations_for_current_user_link + ';count=1'
get_myjohndeere_api_collection_values_for_all_pages(oauth_session, myjohndeere_uri = organizations_for_current_user_link_with_count_of_1)

Now we're able to follow the `nextPage` links. We can quickly add another helper function to set the count to 100 for the first page. This is the current maximum page size.

In [0]:
def default_to_count_100(followable_link) :
  return followable_link + ";count=100"

To bring all of this together, lets call for our organization list using a page size of 100 and make sure we get the full response. We can also combine this with the `showLinks` request param to limit ourselves down to just the `machines`, `files`, and `fields` links.

In [0]:
show_links_request_param_for_organization_list = {'showLinks' : 'machines,files,fields'}

organizations_for_current_user_link = current_user_response['links']['organizations']
organizations_for_current_user_link_with_page_size_of_100 = default_to_count_100(organizations_for_current_user_link)
get_myjohndeere_api_collection_values_for_all_pages(oauth_session, organizations_for_current_user_link_with_page_size_of_100, params = show_links_request_param_for_organization_list)

#Using `embed` to optimize data retrieval#

When setting up your app to retrieve data from the MyJohnDeere API, you will often need to retrieve a variety of related information. Many of the individual resources within the MyJohnDeere API support the concept of an '`embed`', which is when related resources are included with the resource being asked for.

A specific example of `embed` being used in practice is how many consumers of the MyJohnDeere API get basic field information. Lets start by using one of the organizations from the list we just retrieved to see which fields are in that organization.

In [0]:
show_field_links_request_param_for_organization_list = {'showLinks' : 'fields'}

organization_list_for_logged_in_user = get_myjohndeere_api_collection_values_for_all_pages(oauth_session, organizations_for_current_user_link_with_page_size_of_100, params = show_field_links_request_param_for_organization_list)
organization_which_has_fields_link = None
for organization in organization_list_for_logged_in_user:
  if 'fields' in organization['links']:
    organization_which_has_fields_link = organization
    break

assert(organization_which_has_fields_link != None), "The logged in user is not a staff member of any organization in which they can see fields"
organization_which_has_fields_link

Lets now follow the `fields` link to see which fields are available to our user within this organization.

In [0]:
organization_field_list_uri = organization_which_has_fields_link['links']['fields']
organization_field_list_uri_with_count_100 = default_to_count_100(organization_field_list_uri)


field_list_for_organization = get_myjohndeere_api_collection_values_for_all_pages(oauth_session, organization_field_list_uri_with_count_100)
field_list_for_organization

So long as your organization had more than zero fields, you should see the `field` object with a variety of links. Many consumers often want to know the `client` and `farm` for a given field so they can show additional details about a field in the UI, as well as want `simplifiedBoundaries` to know where to show the field on a map. 

Lets make the same call as before, but using the `embed` request param to get that data all in one call for the whole field list.

In [0]:
farm_client_activeBoundary_embed_request_param = {'embed' : 'farms,clients,activeBoundary'}

field_list_for_organization_with_embed = get_myjohndeere_api_collection_values_for_all_pages(oauth_session, organization_field_list_uri_with_count_100, params = farm_client_activeBoundary_embed_request_param)
field_list_for_organization_with_embed

As you can see, `embed` allows your application to extremely efficiently retrieve the data your application needs. We actively encourage our Connected Software Companies to use `embed` on many of our most common API requests. Please let us on the John Deere API support know if you see an oppertunity for us to add more `embed` options to the MyJohnDeere API to help optimize your application's common API requests.

#Optimizing your calls with GZip#

It is important to optimize network traffic time as well as overall data consumption in many web applications. One standard optimization many applications use is to encode and compress the HTTP requests and responses. One of the most common methods of achieving this compression that the MyJohnDeere API uses is to use GZip, which you can read more about at https://en.wikipedia.org/wiki/Gzip

Lets make some API calls where we include the appropriate `Accept-Encoding` so that our response can be encoded correctly

In [0]:
MYJOHNDEERE_V3_JSON_AND_GZIP_HEADERS = {'Accept': 'application/vnd.deere.axiom.v3+json', 'Content-Type': 'application/vnd.deere.axiom.v3+json', 'Accept-Encoding' : 'gzip' }

organization_field_list_with_embed_responses_gzip = oauth_session.get(organization_field_list_uri_with_count_100, headers = MYJOHNDEERE_V3_JSON_AND_GZIP_HEADERS, params = farm_client_activeBoundary_embed_request_param)
field_list_response_size_in_byte_with_gzip = len(organization_field_list_with_embed_responses_gzip.content)
print("Response size of of the field list with GZip was " + str(field_list_response_size_in_byte_with_gzip))

#Recommended administration features to add to your integration#

###Using the `currentToken` object to make sure your customers have uninterrupted service###

Access Tokens for the MyJohnDeere API last for one year. It is good practice to know when your tokens will expire and to include in administration screens to remind your customers to renew their tokens before the year is finished. We'll use the 'currentToken' link from the API Catalog to discover our expiration date

In [0]:
current_token_uri = api_catalog_response_with_clean_links['links']['currentToken']
current_token_json_response = get_myjohndeere_api_json_response(oauth_session, current_token_uri)
current_token_json_response

In [0]:
print("Your token will expire on " + current_token_json_response['expirationDate'])

We recommend that you include alerting to your customers within your own application when tokens are about to expire. We have seen alerts at six and then three months work effectively.

###Using the `currentUser` object as a 'foreign key' into the MyJohnDeere API###

While your application likely has an account creation process and therefore has access to information like your customer's name, it is still useful to be able to understand who the user is within John Deere Operations Center. We'll use the 'currentUser' link from the API Catalog to discover some key details about the logged in user.

In [0]:
current_user_uri = api_catalog_response_with_clean_links['links']['currentUser']
current_user_json_response = get_myjohndeere_api_json_response(oauth_session, current_user_uri)
current_user_json_response

The `currentUser` object contains a few key elements that you'll likely find useful. The `userType` attribute lets you know if the logged in user is a John Deere customer or a John Deere dealer, which you can use to give a more customized experience. The `accountName` attribute is the username of the customer within the John Deere system. Your own tooling should keep a record of this whenever you encounter errors that you plan to submit to John Deere's API support team at APIDevSupport@JohnDeere.com. This will enable the support team to help you  fix issues quickly.

#Handling API errors#

###Dealing with 500 responses###

Error handling is a key part of any rebust system. At John Deere, we are striving to have a system with the maximum possible uptime. However, inevitably your application will encounter a `500` response, or a server failure, from the John Deere servers. You can learn more about HTTP status codes from wikipedia at https://en.wikipedia.org/wiki/List_of_HTTP_status_codes.

We recommend that you add exponential backup logic to your MyJohnDeere API requests to try again later whenever you encounter a 500. For this code section, we'll use [httpstat.us](https://httpstat.us/) as a helpful tool to automatically generate 500 errors for us to encounter.

In [0]:
uri_that_always_generates_500_errors = "https://httpstat.us/500"
oauth_session.get(uri_that_always_generates_500_errors)

Now we'll add some code to retry when we encounter a 500. We will also import some libraries for dealing with time.

In [0]:
import time
import datetime

def is_status_code_a_server_error(http_status_code):
  return http_status_code >= 500 and http_status_code < 600

def print_current_time_to_console():
  print("Making a call at " + str(datetime.datetime.now()))

def call_api_with_500_retry_logic(oauth_session, uri_to_call):
  print_current_time_to_console()
  timer_in_seconds = 1
  api_response = oauth_session.get(uri_to_call)
  while(is_status_code_a_server_error(api_response.status_code) and timer_in_seconds < 30):
    time.sleep(timer_in_seconds)
    timer_in_seconds = timer_in_seconds * 2
    print_current_time_to_console()
    api_response = oauth_session.get(uri_to_call)
  print("Unable to succeed in the API call. This would be a good point to log the exact API call made, the username of the caller, as well as the date")

In [0]:
call_api_with_500_retry_logic(oauth_session, uri_that_always_generates_500_errors)

###Dealing with 429 responses###

The MyJohnDeere API has built-in protections against clients that overwhelm the MyJohnDeere API servers with requests. These same protections are also used when the MyJohnDeere API server itself is under unusually high load. The protection that is used is the `429` response, which you can learn more about at https://tools.ietf.org/html/rfc6585 and well as MyJohnDeere API's specific implemetation at https://developer.deere.com/#!documentation&doc=myjohndeere%2F429.htm&anchor=

To set up the coding example, we'll start by setting up a `mock` to make sure a request to a mock version of `API Catalog` returns backs the `429` response.



In [0]:
!pip install requests_mock

In [0]:
import requests
import requests_mock

mock_api_catalog_uri = 'mock://sandboxapi.deere.com/platform/'

session = requests.Session()
adapter = requests_mock.Adapter()
session.mount('mock', adapter)

retry_after_two_seconds_header = {'Retry-After' : '2'}
adapter.register_uri('GET', mock_api_catalog_uri, status_code = 429, headers = retry_after_two_seconds_header)

With 500 errors, the exponential backup was the recommended approach in order to eventually succeed without overwhelming the servers. With the 429 response, the MyJohnDeere API server gives a specific recommendation for the number of seconds to wait before attempting to retry. We will use that response header in order to setup our sleep timers.

In [0]:
import time
import datetime

def is_status_code_429(http_status_code):
  return http_status_code == 429

def print_current_time_to_console():
  print("Making a call at " + str(datetime.datetime.now()))
  
def call_api_with_429_retry_logic(session, uri_to_call):
  print_current_time_to_console()
  attempts = 0
  api_response = session.get(uri_to_call)
  while(is_status_code_429(api_response.status_code) and attempts < 3):
    time.sleep(int(api_response.headers['Retry-After']))
    attempts = attempts + 1
    print_current_time_to_console()
    api_response = session.get(uri_to_call)
  print("For the 429, it can still be useful to have a maximum amount of retrys before considering the API call a failure")

In [0]:
call_api_with_429_retry_logic(session, mock_api_catalog_uri)

#Getting your application to production#

###Application access while in Sandbox###

Applications on developer.deere.com have two major lifecycle phases. One is called "Sandbox", which is when your application is in testing and the number of users of your application is limited. The second is called "Production", which is when the limitation on the number of users is removed. In both cases, your application is interacting with the same users an the same data as is available in the MyJohnDeere Operations Center.

To limit the number of users of an application in Sandbox within the MyJohnDeere API, only "Team" members of an application on developer.deere.com are allowed to complete OAuth. So that you can see same error message, the next cell should be attempted with a John Deere Account that is not a "Team" member of your application

This is a picture showing the "TEAM MEMBERS" of an application on developer.deere.com. Only these team members can complete OAuth for this application while the application is in "Sandbox".

![Team Section on developer.deere.com](https://github.com/JohnDeere/DevelopWithDeere2018-Mannheim/raw/master/Developer%20Track/Best%20Practices/TeamOnDeveloperDeereCom.png)

In [0]:
#This cell should be run as a user that is not a team member of the client specified at the top of the notebook if you're interested in seeing the 403 error message yourself.
other_request_resource_owner_key, other_request_resource_owner_secret, other_complete_authorize_url = get_request_token_information_for_myjohndeere_api()
"Click On " + complete_authorize_url + " to see the attempt to complete OAuth as a non-team member"

Below shows the error message you will get if you attempt to complete OAuth with a John Deere user that is not a team member of your application on developer.deere.com

![403 on OAuth attempt by non-team member](https://github.com/JohnDeere/DevelopWithDeere2018-Mannheim/raw/master/Developer%20Track/Best%20Practices/403ErrorOnOAuthFailure.png)

###Getting to production###

When you feel your application is ready to go to production with the MyJohnDeere API, such as because you're integration is ready for broading user testing or to be publicly available to all, there is two key steps your application must do in order to be granted access to Production.

The John Deere's API support team needs to do an engineering review of your application's API usage. John Deere's API support team will schedule a review, where it is requested that


*   Your application should be used like a real customer would use your application. This can include initial signup steps, potential data creation steps, as well as polling loops
*   We make sure the scheduled time is at a time of overlapping business hours for both teams. If it is not possible, we'll adjust the time to favor your hours
*   John Deere's API support team will review the logs of your application's API usage

If anything critical is noticed in the review, such as potential DDoS attacks or extremely inefficient usage of the APIs, the review will not pass. Otherwise, the engineering review is done.

The John Deere Business Development team needs to get an API usage agreement signed between your software company and John Deere. This agreement goes over all of the APIs required, the allowed maximum usage of those APIs, as well as what behavior is and is not allowed.

Once all of the above steps are complete, production access will be granted. In technical practice, this means that your application will need to change the base uri that is used to communicate with the MyJohnDeere API. The base uri will be 'https://partnerapi.deere.com/platform/ ' instead of 'https://sandboxapi.deere.com/platform/ '.

#Selecting the right organization for a grower#

Most applications connected to the MyJohnDeere API are focused on helping a farmer improve some aspect of farming. While uncommon, farmers can be members of multiple organizations within Operations Center. The following example is potential method that your application can use for John Deere users that are members of multiple organizations. In the second cell, make sure to change the `selected_organization_index` to an organization from the results from the first cell.

In [0]:
current_user_response = get_myjohndeere_api_json_response(oauth_session, current_user_uri)
organizations_for_current_user_uri = current_user_response['links']['organizations']
organizations_for_current_user_uri = default_to_count_100(organizations_for_current_user_uri)
organizations_for_current_user = get_myjohndeere_api_collection_values_for_all_pages(oauth_session, organizations_for_current_user_uri)
organization_index = 0
for organization in organizations_for_current_user:
  print("[" + str(organization_index) + "] " + organization['name'])
  organization_index = organization_index + 1

In [0]:
#Replace the value of selected_organization_index to one of the values listed from the cell above
selected_organization_index = 0

selected_organization = organizations_for_current_user[selected_organization_index]
print("The organization chosen has a name of \"" + selected_organization['name'] + "\" and an id of " + selected_organization['id'])

#Selecting the right organization for a contractor or a John Deere dealer#

Applications that focus on improving a contractors or a John Deere dealer's ability to support their customers often need to look at organizations a bit differently. They need to be able to see now only the organizations of the logged in user but also all the organizations that the logged in user supports. Often times the applications that contractors and John Deere dealers use need to help support multiple organizations at the same time. The following example shows a possible method that your application can use to select organizations for a contractor.

Note - Your application will still need to make later calls for the resources your application needs from each organization. This selection process is just to make sure your application only downloads and uses the data it needs to serve the contractor or John Deere dealer 

In [0]:
api_catalog_response = get_myjohndeere_api_json_response(oauth_session, api_catalog_url)
organizations_uri = api_catalog_response['links']['organizations']
organizations_uri = default_to_count_100(organizations_uri)

organizations_visible_to_logged_in_user = get_myjohndeere_api_collection_values_for_all_pages(oauth_session, organizations_uri)
organization_index = 0
for organization in organizations_visible_to_logged_in_user:
  print("[" + str(organization_index) + "] " + organization['name'])
  organization_index = organization_index + 1

In [0]:
#Replace the values in the array below with the organizations you want to pick from above
selected_organizations_array = [0, 1]

selected_organizations = [organizations_visible_to_logged_in_user[i] for i in selected_organizations_array]
print("The number of organizations chosen was " + str(len(selected_organizations)))